In [8]:
# ! pip install python-dotenv
# ! pip install -qU langchain-google-vertexai
! pip install langchain_groq
! pip install langchain
! pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.1 MB/s eta 0:00:0000:0100:010m
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached marshmallow-3.23.1-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [9]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import FakeEmbeddings

In [10]:
# Load the environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
model_version = "llama-3.3-70b-versatile"


In [13]:
# Create Model Instance and Invoke a message to the model
model = ChatGroq(api_key=GROQ_API_KEY, model=model_version)
# model.invoke("Hello, how are you?")

# Parse the output through the STROutputParser
parser = StrOutputParser()
chain = model|parser
chain.invoke("Hello, how are you?")

"Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or topics you'd like to discuss. How about you? How's your day going so far?"

In [16]:
# Testing the Models ability to recollect
# chain.invoke("Hello, I'm Emmanuel")

from langchain_core.messages import AIMessage, HumanMessage
chain.invoke(
    [
        HumanMessage("Hello,I'm Emmanuel"),
        AIMessage("Hello Emmanuel, it's nice to meet you. Is there something I can help you with or would you like to chat?"),
        HumanMessage("What is my Name?")
    ]
)

'Your name is Emmanuel.'

In [18]:
# Using Langgraph Message Persistence.
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define A Graph
workflow = StateGraph(state_schema=MessagesState)

# Define the Function That calls the model
def call_quaver(state: MessagesState):
    response = chain.invoke(state["messages"])
    return {"messages": response}

#define the single node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_quaver)

# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [19]:
config = {"configurable": {"thread_id": "abc123"}}

In [20]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================ Human Message =================================

Hello Bob, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [25]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Your name is Bob. I've been reminded of it several times in our conversation.


In [23]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================ Human Message =================================

Your name is still Bob. We've only had a brief conversation so far, but I remember you told me your name is Bob.
